In [9]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import GroupKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Directory containing data files
data_dir = r'C:\Users\User\Documents\Lie detect data\AugmentedEEGData'
model_save_dir = r'C:\Users\User\Documents\Lie detect data\Model'
os.makedirs(model_save_dir, exist_ok=True)

# Function to load and label data
def load_data(data_dir):
    X = []
    y = []
    groups = []
    
    for idx, file_name in enumerate(os.listdir(data_dir)):
        if file_name.endswith('.pkl'):
            file_path = os.path.join(data_dir, file_name)
            data = pd.read_pickle(file_path)
            label = 0 if 'lie' in file_name else 1
            X.append(data)
            y.extend([label] * data.shape[0])
            groups.extend([idx] * data.shape[0])  # Use file index as group label
    
    X = np.vstack(X)
    y = np.array(y)
    groups = np.array(groups)
    return X, y, groups

# Load and label data
X, y, groups = load_data(data_dir)

# Define dataset class
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define EEGNet model
class EEGNet(nn.Module):
    def __init__(self, num_classes=2):
        super(EEGNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (1, 63), padding='same')
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.depthwiseConv2d = nn.Conv2d(16, 32, (65, 1), groups=16, padding='same')
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.activation = nn.ELU()
        self.pooling = nn.AvgPool2d((1, 4))
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32 * 65 * 31, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.depthwiseConv2d(x)
        x = self.batchnorm2(x)
        x = self.activation(x)
        x = self.pooling(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_and_evaluate(train_loader, val_loader, fold_idx):
    model = EEGNet(num_classes=2).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    num_epochs = 100
    best_val_loss = float('inf')
    patience = 10
    trigger_times = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
        
        scheduler.step(val_loss)

        
        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trigger_times = 0
            fold_model_path = os.path.join(model_save_dir, f'model_fold_{fold_idx}.pth')
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'val_loss': best_val_loss,
            }, fold_model_path)
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print('Early stopping!')
                break

    return model

# K-Fold Cross Validation
kf = GroupKFold(n_splits=5)
all_labels = []
all_predictions = []
fold_idx = 1

for train_index, val_index in kf.split(X, y, groups):
    print(f'Fold {fold_idx}')

    # Split data
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Normalize data
    scaler = StandardScaler()
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_train = scaler.fit_transform(X_train)
    X_train = X_train.reshape(-1, 65, 125)

    X_val = X_val.reshape(X_val.shape[0], -1)
    X_val = scaler.transform(X_val)
    X_val = X_val.reshape(-1, 65, 125)

    # Create datasets
    train_dataset = EEGDataset(X_train, y_train)
    val_dataset = EEGDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

    model = train_and_evaluate(train_loader, val_loader, fold_idx)

    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(y_batch.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    fold_idx += 1

# Calculate additional metrics
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)
auc = roc_auc_score(all_labels, all_predictions)
conf_matrix = confusion_matrix(all_labels, all_predictions)

print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1}, AUC: {auc}')
print('Confusion Matrix:')
print(conf_matrix)

Fold 1
Epoch 1/100, Loss: 0.4979856510957082, Validation Loss: 0.7786369025707245, Validation Accuracy: 0.5645161290322581
Epoch 2/100, Loss: 0.283914382259051, Validation Loss: 0.856267899274826, Validation Accuracy: 0.6827956989247311
Epoch 3/100, Loss: 0.23880709211031595, Validation Loss: 1.0163308382034302, Validation Accuracy: 0.6397849462365591
Epoch 4/100, Loss: 0.21426720668872198, Validation Loss: 1.0707660615444183, Validation Accuracy: 0.7204301075268817
Epoch 5/100, Loss: 0.19454770038525263, Validation Loss: 1.0881497859954834, Validation Accuracy: 0.7419354838709677
Epoch 6/100, Loss: 0.1817438304424286, Validation Loss: 1.1086478233337402, Validation Accuracy: 0.7419354838709677
Epoch 7/100, Loss: 0.1844414696097374, Validation Loss: 1.0990611910820007, Validation Accuracy: 0.7580645161290323
Epoch 8/100, Loss: 0.17792331178983053, Validation Loss: 1.120290756225586, Validation Accuracy: 0.7419354838709677
Epoch 9/100, Loss: 0.17344875385363898, Validation Loss: 1.09922